In [ ]:
import numpy as np
from scipy.stats import wasserstein_distance
from itertools import combinations

import matplotlib.pyplot as plt
import torch

import numpy as np
from scipy.stats import wasserstein_distance
from itertools import combinations

import matplotlib.pyplot as plt
import torch
from torch import nn

In [ ]:
import sys
sys.path.append('../GNN_analysis')

In [ ]:
from sims import *

In [ ]:
layers = ['final_layer1_b1','final_layer1_b2','final_layer2_b1','final_layer2_b2','final_layer3_b1','final_layer3_b2','final_layer4_b1','final_layer4_b2','final_fc']

In [ ]:
nsa_criterion = NSALoss(mode='raw')
rtd_criterion = RTDLoss(dim=1, engine='ripser')

In [ ]:
fv_path = 'output/'
a="final_layer1_b1"
temp = torch.load(fv_path+a+".pt")
num_data = temp.shape[0]

In [ ]:
# Iterate over models
heatmap_holder = {}
nsa_sample_indices = np.random.choice(range(num_data),4000, replace=False)
rtd_sample_indices = np.random.choice(range(num_data),400, replace=False)

cka_heatmap = np.empty((len(layers),len(layers)))
rtd_heatmap = np.empty((len(layers),len(layers)))
nsa_heatmap = np.empty((len(layers),len(layers)))

fv_path = 'output/'
for i in range(len(layers)):
    for j in range(i,len(layers)):
        A_data = torch.load(fv_path+layers[i]+".pt")
        B_data = torch.load(fv_path+layers[j]+".pt")
        X = A_data.view(A_data.shape[0],-1)
        Y = B_data.view(B_data.shape[0],-1)
        print(layers[i],X.shape)
        print(layers[j],Y.shape)
        mean_x = torch.mean(X, axis=0)
        mean_y = torch.mean(Y, axis=0)
        X = X - mean_x
        Y = Y - mean_y
        print("Calculating NSA")
        nsa_heatmap[i,j]=nsa_criterion(X[nsa_sample_indices],Y[nsa_sample_indices])
        nsa_heatmap[j,i]=nsa_heatmap[i,j]
        
        print("Calculating CKA")
        cka_heatmap[i,j]=cka(gram_linear(X.numpy()),gram_linear(Y.numpy()))
        cka_heatmap[j,i]=cka_heatmap[i,j]

        print("Calculating RTD")
        rtd_heatmap[i,j]=rtd(X[rtd_sample_indices],Y[rtd_sample_indices], n_runs=1)
        rtd_heatmap[j,i] = rtd_heatmap[i,j]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
output_directory = f'heatmaps/'
!mkdir -p $output_directory

sns.set()
metric_name = 'NSA'
model_name = 'ResNet-18'
plt.figure(figsize=(10, 8))  # Adjust the figure size as needed
if metric_name=='CKA':
    heatmap_data = 1 - heatmap_data
    metric_name = "CKA'"
annot_kwargs = {"size": 30}  # Adjust the fontsize as needed        
ax = sns.heatmap(nsa_heatmap, annot=False, annot_kws=annot_kwargs, cbar=True)  # Modify cmap and formatting as needed

cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=30) 

# plt.title(f'{metric_name} Sanity Test Heatmap', fontsize=23)
plt.xlabel(f'{model_name} layers', fontsize=30)
plt.ylabel(f'{model_name} layers', fontsize=30)
plt.xticks(fontsize=45)
plt.yticks(fontsize=45)

# Save the heatmap plot as an image
#plt.show()
output_filename = f'{metric_name}_heatmap.png'
output_path = os.path.join(output_directory, output_filename)
plt.savefig(output_path, bbox_inches='tight')
plt.close()  # Close the plot to release resources

print(f'Saved: {output_path}')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
output_directory = f'heatmaps/'
!mkdir -p $output_directory

sns.set()
metric_name = 'RTD'
model_name = 'ResNet-18'
plt.figure(figsize=(10, 8))  # Adjust the figure size as needed
if metric_name=='CKA':
    heatmap_data = 1 - heatmap_data
    metric_name = "CKA'"
annot_kwargs = {"size": 30}  # Adjust the fontsize as needed        
ax = sns.heatmap(rtd_heatmap, annot=False, annot_kws=annot_kwargs, cbar=True)  # Modify cmap and formatting as needed

cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=30) 

plt.title(f'{metric_name} Sanity Test Heatmap', fontsize=23)
plt.xlabel(f'{model_name} layers', fontsize=30)
plt.ylabel(f'{model_name} layers', fontsize=30)
plt.xticks(fontsize=45)
plt.yticks(fontsize=45)

# Save the heatmap plot as an image
#plt.show()
output_filename = f'{metric_name}_heatmap.png'
output_path = os.path.join(output_directory, output_filename)
plt.savefig(output_path, bbox_inches='tight')
plt.close()  # Close the plot to release resources

print(f'Saved: {output_path}')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
output_directory = f'heatmaps/'
!mkdir -p $output_directory

sns.set()
metric_name = 'CKA'
model_name = 'ResNet-18'
plt.figure(figsize=(10, 8))  # Adjust the figure size as needed
if metric_name=='CKA':
    heatmap_data = 1 - cka_heatmap
    metric_name = "CKA'"
annot_kwargs = {"size": 30}  # Adjust the fontsize as needed        
ax = sns.heatmap(heatmap_data, annot=False, annot_kws=annot_kwargs, cbar=True)  # Modify cmap and formatting as needed

cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=30) 

#plt.title(f'{metric_name} Sanity Test Heatmap', fontsize=23)
plt.xlabel(f'{model_name} layers', fontsize=30)
plt.ylabel(f'{model_name} layers', fontsize=30)
plt.xticks(fontsize=45)
plt.yticks(fontsize=45)

# Save the heatmap plot as an image
#plt.show()
output_filename = f'{metric_name}_heatmap.png'
output_path = os.path.join(output_directory, output_filename)
plt.savefig(output_path, bbox_inches='tight')
plt.close()  # Close the plot to release resources

print(f'Saved: {output_path}')